In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("Apple_stock_data.csv")
df['date'] = pd.to_datetime(df['Date'])  # Convert Date column to datetime
df.set_index('date', inplace=True)


: 

In [ ]:

# Compute Indicators
df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
df['EMA_200'] = df['Close'].ewm(span=200, adjust=False).mean()

# RSI calculation using a standard method
def compute_RSI(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = compute_RSI(df['Close'], 14)

# Use previous period's values for breakout indicators
df['5D_High'] = df['High'].rolling(5).max().shift(1)
df['5D_Low'] = df['Low'].rolling(5).min().shift(1)
df['20D_Vol_Avg'] = df['Volume'].rolling(20).mean().shift(1)


In [ ]:
# Define Entry Signals
df['signals'] = 0
long_condition = (
    (df['EMA_50'] > df['EMA_200']) &
    (df['Close'] > df['EMA_50']) & (df['Close'] > df['EMA_200']) &
    (df['RSI'] > 55) &
    (df['Volume'] > df['20D_Vol_Avg'])
)
df.loc[long_condition, 'signals'] = 2

short_condition = (
    (df['EMA_50'] < df['EMA_200']) &
    (df['Close'] < df['EMA_50'])  & (df['Close'] < df['EMA_200']) &
    (df['RSI'] < 45) &
    (df['Volume'] > df['20D_Vol_Avg'])
)
df.loc[short_condition, 'signals'] = -2


In [ ]:
# Backtesting Logic with Exit Conditions and Auto-Flipping Positions
capital = 1000  # Initial capital
balance = capital
position = 0      # 0 means no position, 1 for long, -1 for short
entry_price = 0
returns = []
trade_entries = []  # (date, price, type)
trade_exits = []    # (date, price, type)

for i in range(len(df)):
    row = df.iloc[i]
    current_date = df.index[i]
    
    if position == 0:
        # Check for entry signals when no position is held
        if row['signals'] == 2:
            position = 1
            entry_price = row['Close']
            trade_entries.append((current_date, entry_price, 'Long'))
        elif row['signals'] == -2:
            position = -1
            entry_price = row['Close']
            trade_entries.append((current_date, entry_price, 'Short'))
    else:
        # If a position is open, check for exit conditions and flip position
        if position == 1:
            # For a long position, exit if RSI falls below 50
            if row['RSI'] < 50:
                ret = (row['Close'] - entry_price) / entry_price
                returns.append(ret)
                balance *= (1 + ret)
                trade_exits.append((current_date, row['Close'], 'Long Exit'))
                # Flip to a short position at the same closing price
                position = -1
                entry_price = row['Close']
                trade_entries.append((current_date, entry_price, 'Short'))
        elif position == -1:
            # For a short position, exit if RSI rises above 50
            if row['RSI'] > 50:
                ret = (entry_price - row['Close']) / entry_price
                returns.append(ret)
                balance *= (1 + ret)
                trade_exits.append((current_date, row['Close'], 'Short Exit'))
                # Flip to a long position at the same closing price
                position = 1
                entry_price = row['Close']
                trade_entries.append((current_date, entry_price, 'Long'))

# If a position is still open at the end, close it at the final price.
if position != 0:
    last_price = df['Close'].iloc[-1]
    last_date = df.index[-1]
    if position == 1:
        ret = (last_price - entry_price) / entry_price
        trade_exits.append((last_date, last_price, 'Long Exit'))
    elif position == -1:
        ret = (entry_price - last_price) / entry_price
        trade_exits.append((last_date, last_price, 'Short Exit'))
    returns.append(ret)
    balance *= (1 + ret)
    position = 0


In [ ]:
#  Plotting the Price and Trade Signals
plt.figure(figsize=(12,6))
plt.plot(df.index, df['Close'], label='Stock Price', color='blue')
# Mark trade entries and exits
for entry in trade_entries:
    marker = '^' if entry[2] == 'Long' else 'o'
    color = 'green' if entry[2] == 'Long' else 'orange'
    plt.scatter(entry[0], entry[1], marker=marker, color=color, s=100, label=entry[2])
for exit in trade_exits:
    plt.scatter(exit[0], exit[1], marker='v', color='red', s=100, label=exit[2])
plt.title("Stock Price with Entry, Exit, and Position Flip Signals")
plt.legend(loc='best')
plt.show()

# Backtesting Results
print(f"Final Balance: {balance:.2f}")
print(f"Total Returns: {sum(returns) * 100:.2f}%")
print(f"Number of Trades: {len(returns)}")
